# Create meeting minutes from an Audio file
This is a google colab notebook

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 gradio

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
#config to quantize the Llamma3.1 from 32fp to nf4
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
#Load the tokenizer and Llama quantized model
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
def audio_stt(audio_file):
  audio_file = open(audio_file, "rb")
  transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
  print(transcription)
  return transcription

In [ ]:
# Create prompt for the OPENAI to get MOM
def create_prompt(transcription):
  system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
  user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt}
    ]
  return messages



In [ ]:
def create_mom(audio_file):
  transcription = audio_stt(audio_file)
  messages = create_prompt(transcription)
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
  streamer = TextStreamer(tokenizer)
  outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
  return tokenizer.decode(outputs[0])

In [ ]:
import gradio as gr

# Gradio interface
iface = gr.Interface(
    fn=create_mom,
    inputs=gr.Audio(type="filepath", label="Upload Audio"),
    outputs=gr.Markdown(label="Transcription"),
    title="Create MOM from Audio",
    description="Upload an audio file to transcribe and let AI create meeting minutes."
)

iface.launch()